In [3]:
import geodist
import web_scrapping
import tratamento_base_climatica
from pandas import DataFrame

In [4]:
def recuperar_tuplas_somente_de_aeroportos(soup, caminho_para_trs_da_table):
    tuplas = web_scrapping.recuperar_data_frame_de_tuplas(soup, caminho_para_trs_da_table)
    tuplas = tuplas.loc[tuplas['ICAO:'] != '-']
    return tuplas

In [5]:
dataframe_aeroportos = recuperar_tuplas_somente_de_aeroportos('resource/html_geolocation_aeroportos.html', 'table.tabl tr')

In [6]:
dataframe_estacao_climatica = tratamento_base_climatica.recuperar_data_frame_de_tuplas('resource/climahistorico/')

In [20]:
df_resultante = pd.merge(dataframe_aeroportos, dataframe_estacao_climatica, left_on = 'WMO', right_on = 'Estacao', how = 'left join')

,Altitude:,Estação:,ICAO:,Latitude:,Longitude:,País:,WMO:
760,586,TAUBATE (BASE),SBTA,-23.03,-45.5,BZ,-
761,,CACHIMBO-IN-PAR,SBXI,-9.35,-54.88,BZ,-
762,,NATAL,SBSG,-5.77,-35.37,BZ,-
763,,PORTO URUCU,SBUY,-4.88,-65.35,BZ,-
764,NaN,INPE,Gempak Metar,NaN,NaN,Exportar o arquivo nas versões:,Gempak Synop


In [ ]:
df_resultante